Importation des différentes librairies nécessaires au projet

In [98]:
#!/usr/bin/env python
# -*- coding: utf-8 -*-

__author__ = "Mathilde GAUTEUR, Joseph ARIAS"
__copyright__ = "Copyright 2023, Projet InPoDa"
__credits__ = ["Mathilde GAUTEUR", "Joseph ARIAS"]
__license__ = "GPL"
__maintainer__ = "Joseph ARIAS"
__email__ = "joseph.arias@ens.uvsq.fr"
__status__ = "Development"

import json as js
import time
import geopy.exc
from textblob import TextBlob
import random
import folium
from collections import Counter
from geopy.geocoders import Nominatim
import regex as re
import plotly.express as px
import pandas as pd
import os
import emoji
import gradio as gr
import operator
import threading

Différentes variable permettant le bon fonctionnement du projet

In [99]:
avancement_map = 0 # devient 1 lorsque la géolocalisation des tweets se lance
fin_map = 0 # devint 1 lorsque le thread s'occupant de la carte recensant les lieux de publications des tweets est fini
analyse_finished = 0 # devient 1 lorsque l'ensemble des tweets du fichier json de départ ont été converti en objet puis analysé
file_path = 'aitweets.json' # chemin du fichier à analyser

Création d'une classe tweet afin de faciliter la manipulation et l'analyse des tweets fournit.

In [100]:
class Tweet:
    used_hashtag = {}  # dictionnaire avec le nom du hashtag en clé et la liste des tweets contenant le hashtag en
    # valeur
    used_hashtag_sorted = []
    user_mentioned = {}  # dictionnaire avec le nom de l'utilisateur en clé et la liste des tweets mentionnant
    # l'utilisateur en valeur
    user_mentioned_sorted = []
    tweets_of_users = {}  # dictionnaire avec le nom de l'utilisateur en clé et la liste des tweets de l'utilisateur en
    # valeur
    tweets_of_users_sorted = []
    tweets_users = []
    tweets_polarity = [0, 0, 0]  # avec en indice 0 le nombre de tweets négatifs, en indice 1 le nombre de tweets
    # neutres et en indice 2 le nombre de tweets positifs
    tweets_objectivity = [0, 0]  # avec en indice 0 le nombre de tweets objectifs et en indice 1 le nombre de tweets
    # subjectifs
    topics = {}
    topics_sorted = []
    tweets_localization = []  # liste de toutes les localisations des tweets
    tweets_time = {str(i).zfill(2): 0 for i in range(24)}  # création des clés dans le dictionnaire dans l'ordre
    # croissant pour améliorer le graphique de la fonction visualize_time
    compass = []
    specific_user = {}
    specific_hashtag = {}
    mentioned_by_user = {}
    hashtag_by_user = {}
    retweets = {}
    retweets_sorted = []
    languages = {}
    languages_sorted = []
    nb_tweets = 0

    def __init__(self, tweet: dict):

        if not isinstance(tweet, dict):
            print(f'Un élément n\'est pas un dictionnaire et n\'a donc pas pu être analysé: {tweet}')
        else:
            required_keys = ['id', 'AuthorLocation', 'CreatedAt', 'RetweetCount', 'TweetLanguage', 'TweetText']
            missing_keys = [key for key in required_keys if key not in tweet]
            if missing_keys:
                print(f"Attention : Le dictionnaire {tweet} ne contient pas toutes les clés requises ! \n"
                      f"Clé{'s' if len(missing_keys) > 1 else ''} manquante{'s' if len(missing_keys) > 1 else ''} : "
                      f"{', '.join(missing_keys)}")
            else:
                tweet['Hashtags'] = []
                tweet['Mentions'] = []
                tweet['Topics'] = []
                tweet['Polarity'] = ''
                tweet['Subjectivity'] = ''
                self.id = tweet['id']
                self.localization = tweet['AuthorLocation']
                self.date = tweet['CreatedAt']
                self.rt = int(tweet['RetweetCount'])
                self.langue = tweet['TweetLanguage']
                self.texte = tweet['TweetText']
                self.hashtag = tweet['Hashtags']
                self.mention = tweet['Mentions']
                self.polarity = tweet['Polarity']
                self.subjectivity = tweet['Subjectivity']
                self.topics = tweet['Topics']
                self.author = tweet['Author']
                Tweet.retweets[self.texte] = self.rt
                self.extract_car('#')  # extraction des hashtags utilisés dans le tweet
                self.extract_car('@')  # extraction des utilisateurs mentionnés dans le tweet
                self.analyze_sentiment()  # analyse du sentiment du tweet (négatif, neutre ou positif) et la
                # subjectivité
                self.extract_topics()  # extraction des topics du tweet
                self.list_tweet_by_author()  # ajout du tweet au dictionnaire tweets_of_users avec en clé l'auteur du
                # tweet
                self.analyse_language()
                tweet['Polarity'] = self.polarity
                tweet['Subjectivity'] = self.subjectivity
                tweet['Topics'] = self.topics
                tweet['Mentions'] = self.mention
                tweet['Hashtags'] = self.hashtag
                if self.localization != "":
                    Tweet.tweets_localization.append(self.localization)
                Tweet.tweets_time[self.date[11:13]] += 1
                Tweet.nb_tweets += 1

    @staticmethod
    def instantiate_from_file(filepath="aitweets.json"):
        """ Fonction qui instancie les tweets présents dans un fichier json
        """
        global analyse_finished
        if filepath == '':
            filepath = 'aitweets.json'
        data = open(filepath, 'r', encoding='UTF-8')
        list_tweets = [js.loads(line) for line in data]

        # Noms d'utilisateurs qui seront ajoutés aux tweets afin de mieux répondre aux questions du projet étant donné
        # qu'aucun nom d'utilisateur n'est fourni (certains noms sont plus susceptibles d'apparaître souvent pour
        # rendre l'analyse des données plus intéressante)
        users_names = [*['@The_Fiend'] * 18, *['@Bray_Wyatt'] * 66, *['@Shinsuke_Nakamura'] * 11, *['@Cory'] * 20,
                       '@Chumlee', '@Liklenb', '@Jean_Valjean', '@Martin', '@Dupont', *['@IainLJBrown'] * 100,
                       *['@Paula_Piccard'] * 50, *['@nigewillson'] * 25, *['@machinelearnTec'] * 15, '@Karl_Marx',
                       *['@akbarth3great'] * 15, '@JoshuaBarbeau', '@sankrant', '@machinelearnflx', '@SpirosMargaris',
                       *['@Datascience__'] * 30, *['@Charles_Henry'] * 38, *['@UEYvelines'] * 78,
                       *['@unionetudiante_'] * 99, *['@la_classe_ouvriere'] * 16, *['@ogcnice'] * 6, '@Utah',
                       '@chachat']

        for i in range(len(list_tweets)):
            list_tweets[i]['Author'] = random.choice(users_names)
        try:
            for tweet in list_tweets:
                tweet['TweetText'] = emoji.demojize(tweet['TweetText'])
                tweet['TweetText'] = ''.join(re.split(':[^:]+:', tweet['TweetText']))
        except KeyError:
            print('Pas de texte associé au tweet. Les données fournit ne possèdentprobablement pas la bonne structure')
        for tweet in list_tweets:
            Tweet(tweet)
        Tweet.fill_zone_atterrissage(filepath, list_tweets)

        Tweet.used_hashtag_sorted = sorted(Tweet.used_hashtag.items(),
                                           key=operator.itemgetter(1), reverse=True)  # création d'une liste
        # des hashtags présents dans les tweets analysés triée par ordre décroissant du nombre d'apparitions du
        # hashtag
        Tweet.user_mentioned_sorted = sorted(Tweet.user_mentioned.items(),
                                             key=operator.itemgetter(1),
                                             reverse=True)  # création d'une liste
        # des utilisateurs mentionnés dans les tweets analysés triée par ordre décroissant du nombre de mentions
        # de l'utilisateur
        Tweet.tweets_of_users_sorted = dict(sorted(Tweet.tweets_of_users.items(), key=lambda x: len(x[1]),
                                                   reverse=True))  # création d'une liste des utilisateurs triée
        # par ordre décroissant du nombre de tweets de l'utilisateur
        Tweet.tweets_users = [(key, len(tweets)) for key, tweets in Tweet.tweets_of_users_sorted.items()]
        Tweet.topics_sorted = sorted(Tweet.topics.items(), key=operator.itemgetter(1), reverse=True)
        Tweet.retweets_sorted = sorted(Tweet.retweets.items(), key=operator.itemgetter(1), reverse=True)
        Tweet.languages_sorted = sorted(Tweet.languages.items(), key=operator.itemgetter(1), reverse=True)
        for user in Tweet.hashtag_by_user:
            Tweet.hashtag_by_user[user] = list(set(Tweet.hashtag_by_user[user]))
        for user in Tweet.mentioned_by_user:
            Tweet.mentioned_by_user[user] = list(set(Tweet.mentioned_by_user[user]))
        analyse_finished = 1

    @staticmethod
    def fill_zone_atterrissage(filepath, list_tweets):
        """Remplit la zone d'atterrissage avec tous les tweets de la liste de tweets."""
        new_name = os.path.basename(filepath)
        new_name = os.path.splitext(new_name)[0]

        Tweet.reset_zone_atterrissage(new_name)
        file = open(f'zone_atterrissage_{new_name}.json', 'a')
        for tweet in list_tweets:
            js.dump(tweet, file)
            file.write('\n')
        file.close()
        return f'zone_atterrissage_{new_name}.json'

    @staticmethod
    def reset_zone_atterrissage(name):
        """Supprime tout le text du fichier zone_atterrissage.txt."""
        open(f'zone_atterrissage_{name}.json', "w").close()

    def analyse_language(self):
        langue_correspondance = {
            'en': 'anglais',
            'fr': 'français',
            'und': 'non-défini',
            'ja': 'japonais',
            'es': 'espagnol',
            'da': 'danois',
            'ro': 'roumain',
            'pt': 'portugais',
            'ko': 'coréen',
            'de': 'allemand',
            'in': 'maori',
            'it': 'italien',
            'ar': 'arabe',
            'fa': 'persan',
            'ca': 'catalan',
            'fi': 'finnois'
        }
        self.langue = langue_correspondance[self.langue]
        try:
            Tweet.languages[self.langue] += 1
        except KeyError:
            Tweet.languages[self.langue] = 1

    def extract_car(self, car: str):
        """Fonction qui extrait les hashtags utilisés ou les utilisateurs mentionnés dans le tweet à partir d'une base
        de données et les ajoute à la base de données

        Parameters
        ----------
        car : str
            Le caractère que l'on recherche (# si on cherche les hashtags et @ si on cherche les utilisateurs"
        """

        txt = self.texte
        if car == "@":  # si on veut extraire les utilisateurs mentionnés
            list_car = self.mention
            used_car = Tweet.user_mentioned
            name_car = re.findall(r'@\w+', txt)
            specific = Tweet.specific_user
            for user in name_car:
                try:
                    Tweet.mentioned_by_user[self.author].append(user)
                except KeyError:
                    Tweet.mentioned_by_user[self.author] = [user]
        elif car == "#":  # si on veut extraire les hashtags utilisés
            list_car = self.hashtag
            used_car = Tweet.used_hashtag
            name_car = re.findall(r'#\w+', txt)
            specific = Tweet.specific_hashtag
            for hashtag in name_car:
                try:
                    Tweet.hashtag_by_user[hashtag].append(self.author)
                except KeyError:
                    Tweet.hashtag_by_user[hashtag] = [self.author]
        else:
            return f'Le caractère {car} ne correspond ni aux hashtags ni aux mentions'
        for element in name_car:
            list_car.append(element)
            if element in used_car:
                used_car[element] += 1
            else:
                used_car[element] = 1
            if element in specific:
                specific[element].append(txt)
            else:
                specific[element] = [txt]

    def analyze_sentiment(self):
        """Fonction qui analyse le sentiment d'un tweet (négatif, neutre ou positif)
        """
        sentiment = TextBlob(self.texte).sentiment
        if sentiment[0] < 0:
            self.polarity = 'Negative'
            Tweet.tweets_polarity[0] += 1
        elif sentiment[0] == 0:
            self.polarity = 'Neutral'
            Tweet.tweets_polarity[1] += 1
        elif sentiment[0] > 0:
            self.polarity = 'Positive'
            Tweet.tweets_polarity[2] += 1

        if sentiment[1] <= 0.5:
            self.subjectivity = 'Objective'
            Tweet.tweets_objectivity[0] += 1
        elif sentiment[1] >= 0.5:
            self.subjectivity = 'Subjectif'
            Tweet.tweets_objectivity[1] += 1
        Tweet.compass.append([sentiment[0], sentiment[1]])

    def extract_topics(self):
        equivalence = {'artificial intelligence': ['ai', 'artificialintelligence', 'artificial', 'artif', 'arti', 'art',
                                                   'intoainews', 'intelligence', 'artificial_intelligence',
                                                   'artificia', 'artificialin', 'artificialintellig',
                                                   'artificialintellige', 'artificialintell', 'ainews', 'artificialint',
                                                   'artificialintelli', 'artific', 'ia', 'inteligenciaartificial',
                                                   'artificialintelligen', 'artificialintel', 'artificialinte',
                                                   'arificialinterlligence', 'artificialintelligenc', 'intela',
                                                   'artificialintelligenc', 'ronald_teaches_artificial_intelligence',
                                                   'artificial intelligence', 'ia', 'artifici',
                                                   'artificialintelligencetechnology'],
                       'machine learning': ['machinelearn', 'machinelearning', 'ml', 'deeplearning', 'learning',
                                            'machine', 'dl', 'nlp', 'machinelea', 'datascien', 'machi', 'machinelearni'
                                                                                                        'deeple',
                                            'mach', 'machinelearnin', 'tensorflow', 'deepneuralnetworks',
                                            'machinele', 'neuralnetworks', 'machin', 'machinelear', 'sciketlearn',
                                            'deeplearningframework', '100daysofmlcode', 'machinelearni',
                                            'machine learning', 'deeple'],
                       'data': ['datascience', 'bigdata', 'analytics', 'data', 'datatype', 'datax', 'datas', 'datasc',
                                'bigdat', 'datasci', 'dat', 'hdatasystems'],
                       'programmation': ['python', 'python3', 'programming', '100daysofcode', 'coding', 'javascript',
                                         'java', 'sql', 'code', 'cod', '100da', '100daysof', 'pytho', '100days',
                                         'fullstack', '100day', 'coder', '100daysofcod', 'cloudcomputing', 'prog',
                                         'flutte', 'algorithm', 'programmerlife', 'iot', 'programmation', 'numpy',
                                         'devcommunity']
                       }
        for hashtag in self.hashtag:
            hashtag = hashtag[1::].lower()
            for topic in equivalence:
                if hashtag in equivalence[topic]:
                    hashtag = topic
                self.topics.append(hashtag)

        for word in self.texte.split():
            word = word.lower()
            for topic in equivalence:
                if word in equivalence[topic]:
                    word = topic
                    self.topics.append(word)

        self.topics = list(set(self.topics))

        for topic in self.topics:
            if topic in Tweet.topics:
                Tweet.topics[topic] += 1
            else:
                Tweet.topics[topic] = 1

    def list_tweet_by_author(self):
        """Fonction qui ajoute le tweet à la liste des tweets d'un utilisateur
        """
        try:
            Tweet.tweets_of_users[self.author].append(self.texte)
        except KeyError:
            Tweet.tweets_of_users[self.author] = [self.texte]

In [101]:
Tweet.instantiate_from_file()

Fonction qui retourne tout les utilisateurs utilisant un hashtag particulier

In [102]:
def user_mention_specific_hashtag(hashtag: str):
    if hashtag[0] != '#':
        hashtag = '#' + hashtag
    try:
        i = 1
        for utilisateur in Tweet.hashtag_by_user[hashtag]:
            print(f'Utilisateur n°{i} utilisant {hashtag}: {utilisateur}')
            i += 1
    except KeyError:
        print(f'{hashtag} n\'a jamais été utilisé')

user_mention_specific_hashtag('#AI')
user_mention_specific_hashtag('#machine')
user_mention_specific_hashtag('#chocolat')

Utilisateur n°1 utilisant #AI: @ogcnice
Utilisateur n°2 utilisant #AI: @la_classe_ouvriere
Utilisateur n°3 utilisant #AI: @Dupont
Utilisateur n°4 utilisant #AI: @Charles_Henry
Utilisateur n°5 utilisant #AI: @UEYvelines
Utilisateur n°6 utilisant #AI: @Bray_Wyatt
Utilisateur n°7 utilisant #AI: @unionetudiante_
Utilisateur n°8 utilisant #AI: @IainLJBrown
Utilisateur n°9 utilisant #AI: @nigewillson
Utilisateur n°10 utilisant #AI: @machinelearnTec
Utilisateur n°11 utilisant #AI: @Shinsuke_Nakamura
Utilisateur n°12 utilisant #AI: @Paula_Piccard
Utilisateur n°13 utilisant #AI: @The_Fiend
Utilisateur n°14 utilisant #AI: @akbarth3great
Utilisateur n°15 utilisant #AI: @Datascience__
Utilisateur n°16 utilisant #AI: @Cory
Utilisateur n°1 utilisant #machine: @Charles_Henry
Utilisateur n°2 utilisant #machine: @UEYvelines
Utilisateur n°3 utilisant #machine: @Bray_Wyatt
Utilisateur n°4 utilisant #machine: @unionetudiante_
Utilisateur n°5 utilisant #machine: @IainLJBrown
Utilisateur n°6 utilisant #mach

Fonction qui retourne tous les utilisateurs mentionnés par un utilisateur spécifique.

In [103]:
def users_mention_by_user(user: str):
    if user[0] != '@':
        user = '@' + user
    try:
        i = 1
        for utilisateur in Tweet.mentioned_by_user[user]:
            print(f'Utilisateur n°{i} mentionné par {user}: {utilisateur}')
            i += 1
    except KeyError:
        print(f'{user} n\'a jamais mentionné ou tweeté')

users_mention_by_user('@UEY')
users_mention_by_user('@Chumlee')
users_mention_by_user('@Charlou')

@UEY n'a jamais mentionné ou tweeté
Utilisateur n°1 mentionné par @Chumlee: @KingSazeeb
@Charlou n'a jamais mentionné ou tweeté


Fonction qui fait apparaître un graphique montrant les top k (marche pour utilisateurs, hashtags, utilisateurs mentionnés, topics)

In [104]:
def get_top(list_used=Tweet.used_hashtag_sorted, k=10):
    """Top k hashtags ou Top k utilisateurs mentionnés

    Fonction qui affiche les top k hashtags ou les tops k utilisateurs mentionnés

    Paramètres
    ----------
    list_used : list
        liste des hashtags et leurs occurrences ou liste des mentions d'utilisateurs et leurs occurrences
    k : int
        les k hashtags ou utilisateurs qui reviennent le plus

    """
    name = []
    occurrence = []

    for i in range(0, k):
        try:
            if list_used == Tweet.tweets_users:
                top = 'utilisateur'
                mention = 'tweet'
                x_label = 'Utilisateurs'
                y_label = 'Nombre de tweets'
            elif list_used == Tweet.used_hashtag_sorted:
                top = 'hashtag'
                mention = 'occurrence'
                x_label = 'Hashtags'
                y_label = 'Nombre d\'utilisations'
            elif list_used == Tweet.user_mentioned_sorted:
                top = 'utilisateur mentionné'
                mention = 'mention'
                x_label = 'Utilisateurs mentionnés'
                y_label = 'Nombre de mentions'
            elif list_used == Tweet.topics_sorted:
                top = 'sujets de discussions'
                mention = 'mention'
                x_label = 'topics mentionnés'
                y_label = 'Nombre de mentions'
            else:
                return f'La liste {list_used} n\'est pas compatible avec la fonction top'

            print(
                f"Top {i + 1} {top} : {list_used[i][0]} avec {list_used[i][1]} {mention}"
                f"{'s' if list_used[i][1] > 1 else ''}")
            name.append(list_used[i][0])
            occurrence.append(list_used[i][1])
        except IndexError:
            pass
    df = pd.DataFrame({x_label: name, y_label: occurrence})
    fig = px.bar(df, x=x_label, y=y_label, text=y_label, title=f'Top {k} des {top}s',
                 labels={x_label: x_label, y_label: y_label})
    fig.show()
    return fig

get_top(Tweet.tweets_users, 11)
get_top(Tweet.used_hashtag_sorted, 11)
get_top(Tweet.user_mentioned_sorted, 11)
get_top(Tweet.topics_sorted, 11)

Top 1 utilisateur : @IainLJBrown avec 288 tweets
Top 2 utilisateur : @unionetudiante_ avec 273 tweets
Top 3 utilisateur : @UEYvelines avec 226 tweets
Top 4 utilisateur : @Bray_Wyatt avec 176 tweets
Top 5 utilisateur : @Paula_Piccard avec 156 tweets
Top 6 utilisateur : @Charles_Henry avec 107 tweets
Top 7 utilisateur : @Datascience__ avec 81 tweets
Top 8 utilisateur : @nigewillson avec 74 tweets
Top 9 utilisateur : @Cory avec 56 tweets
Top 10 utilisateur : @machinelearnTec avec 52 tweets
Top 11 utilisateur : @The_Fiend avec 51 tweets


Top 1 hashtag : #AI avec 168 occurrences
Top 2 hashtag : #ArtificialIntelligence avec 144 occurrences
Top 3 hashtag : #MachineLearning avec 77 occurrences
Top 4 hashtag : #DataScience avec 47 occurrences
Top 5 hashtag : #ai avec 43 occurrences
Top 6 hashtag : #artificialintelligence avec 25 occurrences
Top 7 hashtag : #BigData avec 21 occurrences
Top 8 hashtag : #5G avec 20 occurrences
Top 9 hashtag : #fintech avec 19 occurrences
Top 10 hashtag : #Python avec 17 occurrences
Top 11 hashtag : #learning avec 17 occurrences


Top 1 utilisateur mentionné : @IainLJBrown avec 112 mentions
Top 2 utilisateur mentionné : @Paula_Piccard avec 49 mentions
Top 3 utilisateur mentionné : @nigewillson avec 24 mentions
Top 4 utilisateur mentionné : @machinelearnTec avec 17 mentions
Top 5 utilisateur mentionné : @akbarth3great avec 17 mentions
Top 6 utilisateur mentionné : @sankrant avec 17 mentions
Top 7 utilisateur mentionné : @machinelearnflx avec 16 mentions
Top 8 utilisateur mentionné : @SpirosMargaris avec 16 mentions
Top 9 utilisateur mentionné : @JoshuaBarbeau avec 16 mentions
Top 10 utilisateur mentionné : @Datascience__ avec 15 mentions
Top 11 utilisateur mentionné : @byLilyV avec 13 mentions


Top 1 sujets de discussions : artificial intelligence avec 1017 mentions
Top 2 sujets de discussions : machine learning avec 228 mentions
Top 3 sujets de discussions : data avec 157 mentions
Top 4 sujets de discussions : programmation avec 93 mentions
Top 5 sujets de discussions : machinelearning avec 89 mentions
Top 6 sujets de discussions : datascience avec 56 mentions
Top 7 sujets de discussions : bigdata avec 27 mentions
Top 8 sujets de discussions : ml avec 26 mentions
Top 9 sujets de discussions : python avec 20 mentions
Top 10 sujets de discussions : fintech avec 20 mentions
Top 11 sujets de discussions : 5g avec 20 mentions


Fonction qui retourne les k tweets les plus retweetés

In [105]:
def top_retweets(k):
    for i in range(0, k):
        print(
            f"Top {i + 1} des tweets les plus retweetés :\n\"{Tweet.retweets_sorted[i][0]}\"\n avec "
            f"{Tweet.retweets_sorted[i][1]} retweet{'s' if Tweet.retweets_sorted[i][1] > 1 else ''}\n")

top_retweets(3)

Top 1 des tweets les plus retweetés :
"RT @ThomasSowell: Ours may become the first civilization destroyed, not by the power of our enemies, but by the ignorance of our teachers a…"
 avec 10327 retweets

Top 2 des tweets les plus retweetés :
"RT @TheTweetOfGod: Artificial intelligence will never be a bigger threat to humanity than natural stupidity."
 avec 6481 retweets

Top 3 des tweets les plus retweetés :
"RT @elleprovocateur What does #Imperial College, the World Economic Forum (#WEF), #Salesforce, #Sinovatio…"
 avec 1660 retweets



Fonction qui retourne l'ensemble des tweets mentionnant un utilisateur spécifique ou l'ensemble des tweets utilisant un hashtag spécifique

In [106]:
def mention_specific(mention: str):
    if mention[0] == '@':
        list_used = Tweet.specific_user
        i = 1
        try:
            for tweet in list_used[mention]:
                print(f'Tweet n°{i} mentionnant {mention}:\n"{tweet}"\n')
                i += 1
        except KeyError:
            print('Cet utilisateur n\'a jamais été mentionné\n')
    elif mention[0] == '#':
        list_used = Tweet.specific_hashtag
        i = 1
        try:
            for tweet in list_used[mention]:
                print(f'Tweet n°{i} utilisant {mention}:\n"{tweet}"\n')
                i += 1
        except KeyError:
            print('Ce hashtag n\'a jamais été utilisé\n')

mention_specific('#DataS')
mention_specific('@Datascience__')

Tweet n°1 utilisant #DataS:
"RT @Paula_Piccard//t.co/xg5LdUCxwb

#MachineLearning  #DataS…"

Tweet n°2 utilisant #DataS:
"RT @Paula_Piccard//t.co/xg5LdUCxwb

#MachineLearning  #DataS…"

Tweet n°3 utilisant #DataS:
"RT @kunaldchowdhurybackhand_index_pointing_down//t.co/Imf1GbAZev

#MachineLearning #5G #DataS…"

Tweet n°1 mentionnant @Datascience__:
"RT @Datascience__ A Perfect Coupling? https://t.co/5SoU9EbhHG  #ArtificialIntelligence"

Tweet n°2 mentionnant @Datascience__:
"RT @Datascience__//t.co/ysxBNkYney  #ArtificialIntelligence"

Tweet n°3 mentionnant @Datascience__:
"RT @Datascience__//t.co/aaixAipN17  #ArtificialInt…"

Tweet n°4 mentionnant @Datascience__:
"RT @Datascience__//t.co/vszxiTneH4  #ArtificialIntelligence"

Tweet n°5 mentionnant @Datascience__:
"RT @Datascience__//t.co/h55YxdrYsR  #Artif…"

Tweet n°6 mentionnant @Datascience__:
"RT @Datascience__//t.co/OqdP5iNRrR  #ArtificialIntellige…"

Tweet n°7 mentionnant @Datascience__:
"RT @Datascience__//t.co/I4MfJb4rTt  #Artif

Fonction qui retourne le nombe de publication dans où un hashtag spécifique apparait

In [107]:
def number_hashtag(hashtag: str):
    """Nombre de publications par hashtag

    Fonction qui affiche le nombre de publications pour un hashtag donné

    Paramètres
    ----------
    hashtag : str
        nom du hashtag que l'on veut compter

    """
    if hashtag[0] != '#':
        hashtag = '#' + hashtag
    try:   
        dic_hashtag = Tweet.used_hashtag[hashtag]
        print(f"Le hashtag {hashtag} apparaît dans {dic_hashtag} publication"
            f"{'s' if dic_hashtag > 1 else ''}")
    except KeyError:
        print(f'Le hashtag {hashtag} n\'a jamais été utilisé')

number_hashtag('python')

Le hashtag #python apparaît dans 3 publications


Fonction qui retourne tous les tweets d'un utlisateur

In [108]:
def publication_author(author: str):
    """Tweets de l'auteur
    Fonction qui affiche les tweets d'un utilisateur donné

    Paramètres
    ----------
    author : str
        nom de l'auteur dont on veut les tweets

    """
    if author[0] != '@':
        author = '@' + author
    try:
        i = 1
        for tweet in Tweet.tweets_of_users[author]:
            print(f'Tweet {i}:\n"{tweet}"\n')
            i += 1
    except KeyError:
        print('Cet utilisateur n\'a pas tweeté ou n\'existe pas')

publication_author('Chumlee')

Tweet 1:
"RT @KingSazeeb//t.co/tti5TyDuU4"

Tweet 2:
"Real Artificial Intelligence vs. Fake Artificial Intelligence https://t.co/Obuv4DHpsj  #ArtificialIntelligence"

Tweet 3:
"Tech gives new tools for grieving. A clever programmer hooked GPT-3, an AI text generator, to his web site, Project December, and now, for a low fee, you can have text chats with your lost love. #hotscience https://t.co/rnSqbf3W9M"



Fonction qui retourne le nombre de publication d'un utilisateur

In [109]:
def number_publication(author: str):
    if author[0] != '@':
        author = '@' + author
    try:
        print(f'{author} a tweeté {len(Tweet.tweets_of_users[author])} fois')
    except KeyError:
        print('Cet utilisateur n\'a pas tweeté ou n\'existe pas')

number_publication('Chumlee')

@Chumlee a tweeté 3 fois


Fonction affichant un graphique montrant le pourcentage d'utilisation des k langues les plus utlisées (% en fonction des k langues et non l'ensemble des langues)

In [110]:
def languages_plot(k=5):
    top_k_languages = Tweet.languages_sorted[:k]  # Sélectionner les k premières langues

    # Extraire les noms des langues et les compteurs
    languages = [lang[0] for lang in top_k_languages]
    counts = [lang[1] for lang in top_k_languages]

    # Créer le graphique pie chart avec Plotly Express
    fig = px.pie(names=languages, values=counts, title=f'Top {k} des langues les plus utilisées')
    fig.show()

languages_plot()

Fonction affichant un graphique montrant la pondérance des différentes polarité sur l'ensemble des tweets ou le taux de subjectivité/objectivité

In [120]:
def show_pie_chart(list_used: list):
    if list_used == Tweet.tweets_polarity:
        labels = 'Négatif', 'Neutre', 'Positif'
        sizes = Tweet.tweets_polarity
        colors = ['silver', 'lightcoral', 'mediumaquamarine']
        title = 'Représentation de la polarité des tweets'
    elif list_used == Tweet.tweets_objectivity:
        labels = 'Objectif', 'Subjectif'
        sizes = Tweet.tweets_objectivity
        colors = ['cornflowerblue', 'lightcoral']
        title = 'Représentation de l\'objectivité des tweets'
    else:
        return f'La liste {list_used} n\'est pas compatible avec la fonction show_pie_chart'
    df = pd.DataFrame({'labels': labels, 'nombre de tweets': sizes})
    fig = px.pie(df, names='labels', values='nombre de tweets', title=title, color_discrete_sequence=colors)
    fig.show()

show_pie_chart(Tweet.tweets_polarity)
show_pie_chart(Tweet.tweets_objectivity)

Fonction montrant la subjectivité des tweets en fonction de leur polarité

In [112]:
def show_pie_chart2():
    df = pd.DataFrame(Tweet.compass, columns=['Polarité', 'Subjectivité'])
    fig = px.scatter(df, x='Subjectivité', y='Polarité',
                     color='Polarité', color_continuous_scale='RdBu',
                     title='Visualisation des Tweets selon la Subjectivité et la Polarité')
    # Centrage du graphique
    fig.update_layout(xaxis=dict(range=[-0.1, 1.1], zeroline=False, showline=False),
                      yaxis=dict(range=[-1.1, 1.1], zeroline=True, zerolinewidth=1, zerolinecolor='black'),
                      plot_bgcolor='darkseagreen', margin=dict(l=0, r=0, t=0, b=0))

    # Ajout d'une ligne verticale à 0.5 sur l'axe de la subjectivité
    fig.add_shape(type='line', x0=0.5, y0=-1.1, x1=0.5, y1=1.1, line=dict(color='black', width=1))

    # Mise en forme du graphique
    fig.update_traces(marker=dict(size=12, opacity=0.8), selector=dict(mode='markers+text'))
    fig.update_layout(xaxis_title='Subjectivité', yaxis_title='Polarité')
    fig.update_coloraxes(colorbar_title='Polarité')

    # Réglage de l'axe x pour placer la ligne zéro à 0.5 et cacher les valeurs dépassant les seuils
    fig.update_xaxes(showgrid=False, tickvals=[0, 0.5, 1], ticktext=['0', '0.5', '1'], tickmode='array')
    fig.update_yaxes(showgrid=False, tickvals=[-1, 0, 1], ticktext=['-1', '0', '1'], tickmode='array')

    return fig

show_pie_chart2()

Fonction qui crée un fichier .html contenant une carte représentant la localisation de tous les tweets et le nombre de tweets par localisation

In [113]:
def world_map(start=0, end=0, user='Géolocalisation_tweet'):
    print('World Map')
    global fin_map
    global avancement_map
    global analyse_finished
    while analyse_finished != 1:
        time.sleep(1)
    if end == 0:
        end = len(Tweet.tweets_localization)
    debut = time.time()
    geolocator = Nominatim(user_agent=user)
    tweet_coordinates = {}
    avancement_map = 1
    for location in Tweet.tweets_localization[start:end]:
        try:
            location = location.strip()
            if location not in tweet_coordinates:
                location_data = geolocator.geocode(location)
                if location_data:
                    tweet_coordinates[location] = (location_data.latitude, location_data.longitude, location)
            if avancement_map % 10 == 0 or avancement_map == 1:
                print(avancement_map, '/', len(Tweet.tweets_localization))
            avancement_map += 1
        except ConnectionError:
            print(f'Une erreur de connection est survenue lors de la création de la carte avec la localisation '
                  f'suivante: {location}')
            avancement_map += 1
        except TimeoutError:
            print(f'Une erreur de type Timeout est survenue lors de la création de la carte avec la localisation '
                  f'suivante: {location}')
            avancement_map += 1
        except geopy.exc.GeocoderUnavailable:
            print(f'Une erreur est survenue lors de la création de la carte avec la localisation '
                  f'suivante: {location}. Soit la connexion n\'a pas pu être établis avec le service de géolocalisation'
                  f' soit le service n\'est pas disponible')
        except geopy.exc.GeocoderQuotaExceeded:
            print(f'Une erreur est survenue lors de la création de la carte avec la localisation suivante: {location}'
                  f'à cause d\'un trop grand nombre de requête en un certain laps de temps.')
        except Exception as error:
            print(f'Une erreur de type {type(error)}est survenue lors de la création de la carte avec la localisation '
                  f'suivante: {location}')
            avancement_map += 1

    tweet_counts = Counter(Tweet.tweets_localization)
    m = folium.Map()

    for location, count in tweet_counts.items():
        if location in tweet_coordinates:
            lat, lon = tweet_coordinates[location][:2]
            popup_content = f'<p><b>{tweet_coordinates[location][2]}</b></p>' \
                            f'<p>{count} tweet{"s" if count > 1 else ""}' \
                            f' posté{"s" if count > 1 else ""} depuis cet endroit</p>'  # utilisation de html qui
            # est nécessaire étant donné que le fichier contenant la carte est en html
            popup = folium.Popup(popup_content, parse_html=False, max_width=100)
            """icon_image = "icone_twitter.png"
            coeff = 1 + count*0.05
            icon = folium.CustomIcon(
                icon_image,
                icon_size=(50*coeff, 50*coeff),
                icon_anchor=(0, 0),
                popup_anchor=(0, 0)
            )"""
            folium.Marker(location=(lat, lon), popup=popup).add_to(m)

    m.save('tweet_map.html')
    fin = time.time()
    print('temps final:', fin - debut)
    fin_map = 1

Fonction permettant de visualiser le nombre de tweet en fonction de l'heure

In [114]:
def visualize_tweet_time():
    time = list(Tweet.tweets_time.keys())
    nb_tweets = list(Tweet.tweets_time.values())
    df = pd.DataFrame({'Heure': time, 'Nombre de Tweets': nb_tweets})
    plot_time = px.line(df, x='Heure', y='Nombre de Tweets', markers=True, line_shape='linear',
                        labels={'Heure': 'Heure', 'Nombre de Tweets': 'Nombre de Tweets'},
                        title='Nombre de Tweets par Heure')
    return plot_time

visualize_tweet_time()

Interface graphique réalisée avec Gradio (ne marche pas dans le notebook)

In [115]:
"""def start():
    global file_path
    print(file)
    interface_init.close()
    interface.launch()
    Tweet.instantiate_from_file(file_path)
    return {
        welcome_label: gr.Label(visible=False),
        analyze_button: gr.Button(visible=False),
        upload_file_button: gr.UploadButton(visible=False),
        analysis: gr.Radio(visible=True)
    }


def change_r(choice: str):
    if choice == Radio_Choices[-1]:
        return {plot: gr.Plot(visible=False),
                top: gr.Dropdown(visible=False),
                others: gr.Dropdown(visible=False)}

    if choice == Radio_Choices[0]:
        return {top: gr.Dropdown(visible=True),
                others: gr.Dropdown(visible=False),
                plot: gr.Plot(visible=False)}

    if choice == Radio_Choices[1]:
        return {others: gr.Dropdown(visible=True),
                top: gr.Dropdown(visible=False),
                plot: gr.Plot(visible=False)}


def change_top(choice: str):
    if choice == "Top hashtags":
        return {plot: gr.Plot(value=get_top(list_used=Tweet.used_hashtag_sorted, k=10), visible=True)}
    if choice == "Top utilisateurs":
        return {plot: gr.Plot(value=get_top(list_used=Tweet.tweets_users, k=10), visible=True)}
    if choice == "Top utilisateurs mentionnés":
        return {plot: gr.Plot(value=get_top(list_used=Tweet.user_mentioned_sorted, k=10), visible=True)}
    if choice == "Top topics":
        return {plot: gr.Plot(value=get_top(list_used=Tweet.topics_sorted, k=10), visible=True)}


def change_act(choice: str):
    if choice == "Nombre de publications par utilisateur":
        return {publi_user: gr.Textbox(value=number_publication("@Martin"), visible=True)}
    if choice == "Tous les Tweets d'un utilisateur":
        pass
        # return {plot  : gr.Plot(visible=True)} #ajouter value
    if choice == "Tous les utilisateurs mentionnés par un utilisateur":
        pass
        # return {plot  : gr.Plot(visible=True)} #ajouter value


def change_publi(choice: str):
    if choice == "Nombre de publications par topic":
        return {topic: gr.Textbox(visible=True)}
        # return {plot  : gr.Plot(visible=True)} #ajouter value
    if choice == "Nombre de publications par hashtag":
        return {hashtag: gr.Textbox(visible=True)}
        # return {plot  : gr.Plot(visible=True)} #ajouter value
    if choice == "Tous les utilisateurs d'un hashtag":
        return {hashtag: gr.Textbox(visible=True)}
        # return {plot  : gr.Plot(visible=True)} #ajouter value


def change_others(choice: str):
    if choice == "Heures de Tweet":
        return {plot: gr.Plot(value=visualize_tweet_time(), visible=True)}
    if choice == "Polarité/Subjectivité":
        return {plot: gr.Plot(value=show_pie_chart2(), visible=True)}
    if choice == "Répartition mondiale":
        return {carte: gr.HTML(value='tweet_map.html', visible=True)}


def change_slider(value: int):
    val = value if value != 0 else 10
    return {plot: gr.Plot(value=get_top(k=value), visible=True),
            slider: gr.Slider(1, 50, val, step=1, label="Les Top combien voulez-vous voir ?",
                              info="Déplacez le curseur", visible=True, interactive=True)}


def upload_file(file):
    global file_path
    file_path = file
    start()


Radio_Choices = ["Top (4)",
                 "Caractéristiques (3)",
                 "Masquer"]

with gr.Blocks(theme=gr.themes.Soft(neutral_hue='cyan')) as interface:
    title = gr.Label(label="InPoDa", value="InPoDa", color="#00ACEE")
    welcome_label = gr.Label(label="Bonjour", value="Bienvenue sur InPoDa, la plateforme d'analyse de données de"
                                                    "réseaux sociaux.\nVeuillez choisir un fichier à analyser")
    # file_output = gr.File()
    upload_file_button = gr.UploadButton("Cliquez pour choisir le fichier à analyser",
                                         file_types=[".json"], file_count="single", visible=False)
    analyze_button = gr.Button(value="Lancer l'analyse", visible=True)
    carte = gr.HTML(visible=False)
    analysis = gr.Radio(choices=Radio_Choices,
                        value="Masquer",
                        label="Analyses",
                        info="Que voulez-vous analyser ?",
                        visible=False,
                        interactive=True)
    top = gr.Dropdown(choices=["Top hashtags", "Top utilisateurs", "Top utilisateurs mentionnés", "Top topics"],
                      label="Top", info="Veuillez sélectionner l'élément dont vous voulez voir le Top :", visible=False)
    act = gr.Dropdown(choices=["Nombre de publications par utilisateur", "Tous les Tweets d'un utilisateur",
                               "Tous les utilisateurs mentionnés par un utilisateur"],
                      label="Activité", info="Veuillez sélectionner l'activité que vous souhaitez observer :",
                      visible=False)
    publi = gr.Dropdown(choices=["Nombre de publications par topic", "Nombre de publications par hashtag",
                                 "Tous les utilisateurs d'un hashtag"],
                        label="Catégories", info="Veuillez sélectionner les publications que vous voulez analyser :",
                        visible=False)
    others = gr.Dropdown(choices=["Heures de Tweet", "Polarité/Subjectivité", "Répartition mondiale"],
                         label="Autres", info="Veuillez sélectionner ce que vous souhaitez analyser :", visible=False)

    hashtag = gr.Textbox(info="Rentrez le hashtag", visible=False, interactive=True)
    topic = gr.Textbox(info="Rentrez le topic :", visible=False, interactive=True)
    plot = gr.Plot(visible=False)
    slider = gr.Slider(visible=False)
    publi_user = gr.Textbox(info="Quel utilisateur ?", visible=False)
    # slider.change(change_r,inputs=[analysis,slider],outputs=plot)
    analysis.change(change_r, inputs=[analysis], outputs=[top, others, plot])
    top.change(change_top, inputs=[top], outputs=[plot])
    act.change(change_act, inputs=[act], outputs=[publi_user])
    publi.change(change_publi, inputs=[publi], outputs=[hashtag, topic])
    others.change(change_others, inputs=[others], outputs=[plot])
    hashtag.change(change_hashtag, inputs=[hashtag], outputs=[plot])
    topic.change(change_topic, inputs=[topic], outputs=[plot])
    interface.load(change_r, inputs=[analysis], outputs=[plot])
    analyze_button.click(start, outputs=[welcome_label, upload_file_button, analyze_button, analysis])
    upload_file_button.click(upload_file, outputs=[welcome_label, upload_file_button, analyze_button, analysis])


if __name__ == '__main__':
    #Tweet.instantiate_from_file('aitweets.json')
    thread_map = threading.Thread(target=world_map)
    thread_map.start()

    # Partie qui permettrait d'accélérer le processus de world map mais qui est impossible, car geopy refuse un trop
    # grand nombre de requêtes en un certain laps de temps. Pour régler ce problème, on pourrait essayer de passer par
    # un API nœud de TOR avant de changer les ip

    thread_map = []
    for i in range(0, len(Tweet.tweets_localization), 100):
        print('création du thread', i % 100)
        end = min(i + 100,
                  len(Tweet.tweets_localization))
        t = threading.Thread(target=world_map, args=(i, end, get_password(10)))
        thread_map.append(t)
        t.start()
        time.sleep(random.randint(1,5))

    interface.launch()"""

'def start():\n    global file_path\n    print(file)\n    interface_init.close()\n    interface.launch()\n    Tweet.instantiate_from_file(file_path)\n    return {\n        welcome_label: gr.Label(visible=False),\n        analyze_button: gr.Button(visible=False),\n        upload_file_button: gr.UploadButton(visible=False),\n        analysis: gr.Radio(visible=True)\n    }\n\n\ndef change_r(choice: str):\n    if choice == Radio_Choices[-1]:\n        return {plot: gr.Plot(visible=False),\n                top: gr.Dropdown(visible=False),\n                others: gr.Dropdown(visible=False)}\n\n    if choice == Radio_Choices[0]:\n        return {top: gr.Dropdown(visible=True),\n                others: gr.Dropdown(visible=False),\n                plot: gr.Plot(visible=False)}\n\n    if choice == Radio_Choices[1]:\n        return {others: gr.Dropdown(visible=True),\n                top: gr.Dropdown(visible=False),\n                plot: gr.Plot(visible=False)}\n\n\ndef change_top(choice: str):\

Lancement de l'interface graphique (et donc de l'analyse des tweets) et du thread world map

In [116]:
"""with gr.Blocks(theme=gr.themes.Soft()) as demo:
    title = gr.Label(label = "InPoDa", value="InPoda", color="black")
    debut_label = gr.Label(label="",value="Bienvenue sur la plateforme qui va te permettre d'analyser des tweets")
    with gr.Tab("Toutes les langues utilisées"):
        plot = gr.Plot(label="Plot", value=languages_plot)
        demo.load(languages_plot, outputs=[plot])
    with gr.Tab("Lieux"):
        gr.Button("là c'est bon c'est la map ")
        gr.HTML(value='arattaa.html', visible=True)
    with gr.Tab("Hashtags"):
        plot1 = gr.Plot(label="Hashtags", value=get_top)
        demo.load(get_top(Tweet.used_hashtag_sorted), outputs=[plot1])   
        inp = gr.Textbox(placeholder="What is your name?")
        out = gr.Textbox()
        inp.change(mention_specific, inp, out)        
    with gr.Tab("Top 8"):
        gr.Button("Utilisateurs")
        gr.Button("Utilisateurs mentionnés")
        gr.Button("Les hashtags")
        gr.Button("Topic")
    with gr.Tab("Utilisateurs"):
        plot2 = gr.Plot(label = "Utilisateurs", value=mention_specific)
        demo.load(mention_specific(Tweet.user_mentioned_sorted), outputs=[plot2])
    with gr.Tab('Sentiments'):
        gr.Plot(value=show_pie_chart2, visible=True)"""

'with gr.Blocks(theme=gr.themes.Soft()) as demo:\n    title = gr.Label(label = "InPoDa", value="InPoda", color="black")\n    debut_label = gr.Label(label="",value="Bienvenue sur la plateforme qui va te permettre d\'analyser des tweets")\n    with gr.Tab("Toutes les langues utilisées"):\n        plot = gr.Plot(label="Plot", value=languages_plot)\n        demo.load(languages_plot, outputs=[plot])\n    with gr.Tab("Lieux"):\n        gr.Button("là c\'est bon c\'est la map ")\n        gr.HTML(value=\'arattaa.html\', visible=True)\n    with gr.Tab("Hashtags"):\n        plot1 = gr.Plot(label="Hashtags", value=get_top)\n        demo.load(get_top(Tweet.used_hashtag_sorted), outputs=[plot1])   \n        inp = gr.Textbox(placeholder="What is your name?")\n        out = gr.Textbox()\n        inp.change(mention_specific, inp, out)        \n    with gr.Tab("Top 8"):\n        gr.Button("Utilisateurs")\n        gr.Button("Utilisateurs mentionnés")\n        gr.Button("Les hashtags")\n        gr.Button(